In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base

# No need for RunnableConfig or relationship in this schema file
# from langchain_core.runnables.config import RunnableConfig
# from sqlalchemy.orm import relationship

load_dotenv()

# --- IMPORTANT: Updated to point to your new database ---
DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///netcdf_database.db")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()


# Definition of the Ocean Data table
# This class maps directly to the 'ocean_data' table created from your NetCDF file.
class OceanData(Base):
    __tablename__ = "ocean_data"

    # It's good practice to have a primary key, even if not in the original file
    id = Column(Integer, primary_key=True, index=True)
    
    # Coordinates from the NetCDF file
    time = Column(DateTime)
    latitude = Column(Float)
    longitude = Column(Float)
    depth = Column(Float)
    
    # Variables from the NetCDF file
    so = Column(Float)       # Sea Water Salinity
    thetao = Column(Float)   # Sea Water Potential Temperature
    uo = Column(Float)       # Eastward Sea Water Velocity
    vo = Column(Float)       # Northward Sea Water Velocity
    zos = Column(Float)      # Sea Surface Height Above Geoid

# Example of how to create the table if needed (the setup_db.py already did this)
# if __name__ == "__main__":
#     print("Creating database tables...")
#     Base.metadata.create_all(bind=engine)
#     print("Tables created.")

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.orm import sessionmaker
from langgraph.graph import StateGraph, END

# --- Part 1: Database Setup from CSV ---

def setup_database_from_csv(csv_filepath, db_filepath, table_name):
    """
    Reads a CSV file, cleans its data, and loads it into an SQLite database.
    This function will only run if the database file does not already exist.
    """
    if os.path.exists(db_filepath):
        print(f"Database '{db_filepath}' already exists. Skipping creation.")
        return

    print(f"Creating new database '{db_filepath}' from '{csv_filepath}'...")
    try:
        df = pd.read_csv(csv_filepath)
        # Clean column names to be valid SQL identifiers
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
        
        engine = create_engine(f"sqlite:///{db_filepath}")
        df.to_sql(table_name, engine, if_exists='replace', index=False, chunksize=10000)
        
        print(f"✅ Successfully loaded {len(df)} rows into table '{table_name}'.")
    except FileNotFoundError:
        print(f"❌ Error: The CSV file was not found at {csv_filepath}")
    except Exception as e:
        print(f"❌ An error occurred while creating the database: {e}")

# --- Part 2: Agent Configuration ---

load_dotenv()

# Database connection setup
DB_FILE = "ocean_csv_database.db"
DATABASE_URL = os.getenv("DATABASE_URL", f"sqlite:///{DB_FILE}")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Connect to the Ollama model
llm = ChatOllama(model="phi3", temperature=0)

# Define the agent's state
class AgentState(TypedDict):
    question: str
    sql_query: str
    query_result: str
    query_rows: list
    attempts: int
    relevance: str
    sql_error: bool

# --- Part 3: Agent Nodes (Functions) ---

def get_database_schema(engine):
    """Inspects the database and returns its schema as a string."""
    inspector = inspect(engine)
    schema = ""
    for table_name in inspector.get_table_names():
        schema += f"Table: {table_name}\n"
        for column in inspector.get_columns(table_name):
            col_name = column["name"]
            col_type = str(column["type"])
            schema += f"- {col_name}: {col_type}\n"
        schema += "\n"
    print("Retrieved database schema.")
    return schema

class CheckRelevance(BaseModel):
    relevance: str = Field(description="'relevant' or 'not_relevant'.")

def check_relevance(state: AgentState):
    """Checks if the user's question is relevant to the ocean data schema."""
    question = state["question"]
    schema = get_database_schema(engine)
    print(f"Checking relevance of the question: {question}")
    
    system = """You are an assistant that determines if a question is related to the database schema of oceanographic data.
    A question is relevant if it asks about sea temperature, salinity, velocity, coordinates, depth, or time.
    Schema: {schema}
    Respond with a JSON object with a single key 'relevance' and a value of 'relevant' or 'not_relevant'.""".format(schema=schema)
    
    check_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "Question: {question}")])
    parser = PydanticOutputParser(pydantic_object=CheckRelevance)
    relevance_checker = check_prompt | llm.bind(format="json") | parser
    
    relevance = relevance_checker.invoke({"question": question})
    state["relevance"] = relevance.relevance
    print(f"Relevance determined: {state['relevance']}")
    return state

class ConvertToSQL(BaseModel):
    sql_query: str = Field(description="The SQL query for the user's question.")

def convert_nl_to_sql(state: AgentState):
    """Converts the natural language question to a robust SQL query."""
    question = state["question"]
    schema = get_database_schema(engine)
    print(f"Converting question to SQL: {question}")

    system = """You are an expert SQL assistant. Convert natural language questions into robust SQL queries based on the provided schema.
    The table is named 'ocean_data'.
    IMPORTANT: The data exists on a grid. Do not use exact equality checks (e.g., WHERE latitude = 50).
    Instead, find the NEAREST available data point by using an ORDER BY clause with a squared distance calculation and then taking the first result (LIMIT 1).
    Example for latitude 50 and longitude -10: 'SELECT * FROM ocean_data ORDER BY (latitude - 50.0)*(latitude - 50.0) + (longitude - -10.0)*(longitude - -10.0) ASC LIMIT 1;'
    Pay close attention to the column names provided in the schema.
    Schema: {schema}
    Respond with a JSON object with a single key 'sql_query' containing only the SQL query.""".format(schema=schema)
    
    convert_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
    parser = PydanticOutputParser(pydantic_object=ConvertToSQL)
    sql_generator = convert_prompt | llm.bind(format="json") | parser
    
    result = sql_generator.invoke({"question": question})
    state["sql_query"] = result.sql_query
    print(f"Generated SQL query: {state['sql_query']}")
    return state

def execute_sql(state: AgentState):
    """Executes the SQL query and returns the result."""
    sql_query = state["sql_query"].strip()
    session = SessionLocal()
    print(f"Executing SQL query: {sql_query}")
    try:
        result = session.execute(text(sql_query))
        if sql_query.lower().strip().startswith("select"):
            rows = result.fetchall()
            columns = result.keys()
            state["query_rows"] = [dict(zip(columns, row)) for row in rows]
            state["query_result"] = f"Query returned {len(state['query_rows'])} rows."
            print(f"Raw SQL Query Result: {state['query_rows']}")
        else:
            session.commit()
            state["query_result"] = "The action has been successfully completed."
        state["sql_error"] = False
    except Exception as e:
        state["query_result"] = f"Error executing SQL query: {str(e)}"
        state["sql_error"] = True
        print(f"Error executing SQL query: {str(e)}")
    finally:
        session.close()
    return state

def generate_human_readable_answer(state: AgentState):
    """Generates a clear, natural language response from the query results."""
    question = state["question"]
    query_rows = state.get("query_rows", [])
    sql_error = state.get("sql_error", False)
    print("Generating a human-readable answer.")

    system = "You are an assistant who explains database query results about oceanographic data in a clear and concise way."
    
    prompt_text = ""
    if sql_error:
        prompt_text = f"The query for the question '{question}' failed. Explain the error in a simple sentence: {state['query_result']}"
    elif not query_rows:
        prompt_text = f"Based on the question '{question}', formulate a single sentence stating that no data was found for the specified criteria."
    else:
        data_summary = ", ".join([str(row) for row in query_rows])
        prompt_text = f"""The user asked: '{question}'.
        The database returned the following data: {data_summary}.
        Formulate a clear, natural language answer summarizing this data in a single sentence. Mention the specific value requested and the coordinates found.
        For example: 'At the nearest point (latitude X, longitude Y), the depth is Z meters.'"""
    
    generate_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input_text}")])
    human_response = generate_prompt | llm | StrOutputParser()
    answer = human_response.invoke({"input_text": prompt_text})
    
    state["query_result"] = answer
    print("Generated human-readable answer.")
    return state

class RewrittenQuestion(BaseModel):
    question: str = Field(description="The rewritten question.")

def regenerate_query(state: AgentState):
    question = state["question"]
    print("Regenerating the SQL query by rewriting the question.")
    system = """You are an assistant that reformulates a question about oceanographic data to be more precise for SQL conversion.
    Ensure that details like latitude, longitude, and depth are clearly stated.
    Respond with a JSON object with a single key 'question' containing the rewritten question."""
    
    rewrite_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "Original Question: {question}")])
    parser = PydanticOutputParser(pydantic_object=RewrittenQuestion)
    rewriter = rewrite_prompt | llm.bind(format="json") | parser
    
    rewritten = rewriter.invoke({"question": question})
    state["question"] = rewritten.question
    state["attempts"] += 1
    print(f"Rewritten question: {state['question']}")
    return state

# --- Part 4: Graph Definition and Execution ---

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("check_relevance", check_relevance)
workflow.add_node("convert_to_sql", convert_nl_to_sql)
workflow.add_node("execute_sql", execute_sql)
workflow.add_node("generate_human_readable_answer", generate_human_readable_answer)
workflow.add_node("regenerate_query", regenerate_query)

workflow.set_entry_point("check_relevance")

def relevance_router(state: AgentState):
    return "convert_to_sql" if state["relevance"].lower() == "relevant" else END

def execute_sql_router(state: AgentState):
    return "generate_human_readable_answer" if not state.get("sql_error", False) else "regenerate_query"

def check_attempts_router(state: AgentState):
    return "convert_to_sql" if state["attempts"] < 3 else END

workflow.add_conditional_edges("check_relevance", relevance_router)
workflow.add_edge("convert_to_sql", "execute_sql")
workflow.add_conditional_edges("execute_sql", execute_sql_router)
workflow.add_conditional_edges("regenerate_query", check_attempts_router)
workflow.add_edge("generate_human_readable_answer", END)

app = workflow.compile()

# Main execution block
if __name__ == "__main__":
    # --- Step 1: Ensure the database is created from the CSV ---
    csv_file_path = r"C:\Users\apran\Videos\Cin\LIBRARY\SQL Agent for Sensor Data\ocean_data.csv"
    setup_database_from_csv(csv_filepath=csv_file_path, db_filepath=DB_FILE, table_name="ocean_data")

    print("\nSQL Agent for Ocean Data is ready. Ask a question.")
    
    # --- Step 2: Define your question ---
    user_question = "what is the depth for latitude 64.33334 and longitude -3.4166667?"
    
    inputs = {"question": user_question, "attempts": 0}
    final_answer = None
    
    # --- Step 3: Run the agent and get the answer ---
    print("\n--- Agent Execution Log ---")
    for event in app.stream(inputs):
        for key, value in event.items():
            print(f"--- Event: {key} ---")
            print(value)
            # Capture the final answer
            if key == "generate_human_readable_answer":
                final_answer = value.get("query_result")
    
    print("\n" + "="*50)
    print("✅ Final Human-Readable Answer:")
    print("="*50)
    if final_answer:
        print(final_answer)
    else:
        print("Could not generate a final answer from the workflow.")



C:\Users\apran\AppData\Local\Temp\ipykernel_16588\2069348147.py:50: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="phi3", temperature=0)


Creating new database 'ocean_csv_database.db' from 'C:\Users\apran\Videos\Cin\LIBRARY\SQL Agent for Sensor Data\ocean_data.csv'...
✅ Successfully loaded 102393 rows into table 'ocean_data'.

SQL Agent for Ocean Data is ready. Ask a question.

--- Agent Execution Log ---
Retrieved database schema.
Checking relevance of the question: what is the depth for latitude 64.33334 and longitude -3.4166667?
Relevance determined: relevant
--- Event: check_relevance ---
{'question': 'what is the depth for latitude 64.33334 and longitude -3.4166667?', 'attempts': 0, 'relevance': 'relevant'}
Retrieved database schema.
Converting question to SQL: what is the depth for latitude 64.33334 and longitude -3.4166667?
Generated SQL query: SELECT * FROM ocean_data ORDER BY (latitude - 64.33334)*(latitude - 64.33334) + (longitude - -3.4166667)*(longitude - -3.4166667) ASC LIMIT 1;
--- Event: convert_to_sql ---
{'question': 'what is the depth for latitude 64.33334 and longitude -3.4166667?', 'attempts': 0, 'rel